## 卷积运算

在泛函分析中，卷积、旋积或摺积(英语：Convolution)是通过两个函数f 和g 生成第三个函数的一种数学算子，表征函数f 与g经过翻转和平移的重叠部分的面积。

一维卷积：
![pic1](https://ws4.sinaimg.cn/large/006tKfTcly1g13ixznocfj309e02rdfp.jpg)

二维卷积：
![pic2](https://ws1.sinaimg.cn/large/006tKfTcly1g13iyoqvydj30gs02b0so.jpg)

## CNN

局部连接:这个是最容易想到的，每个神经元不再和上一层的所有神经元相连，而只和一小部分神经元相连。这样就减少了很多参数。
权值共享:一组连接可以共享同一个权重，而不是每个连接有一个不同的权重，这样又减少了很多参数。
下采样:可以使用Pooling来减少每层的样本数，进一步减少参数数量，同时还可以提升模型的鲁棒性。

### 反卷积(tf.nn.conv2d_transpose)

传入函数的参数有value，filter，output_shape，strides，padding，data_format和name，最主要的三个参数是value，output_shape和strides。value传入函数是为了确定输入尺寸，output_shape是为了确定输出尺寸，strides就是kernel在vertical和horizon上的步长。
value的格式为[batch, height, width, in_channels]，height和width是用来计算输出尺寸用到的最重要的两个参数，表示输入该层feature map的高度和宽度，典型的NHWC格式；
filter的格式为[height, width, output_channels, input_channels]，务必注意这里的channel数是输出的channel数在前，输入的channel数在后。
output_shape是一个1-D张量，传入的可以是一个tuple或者list，在不指定data_format参数的情况下，格式必须为NHWC。注意：这里的C要与filter中的output_channels保持一致；
strides的格式为一个整数列表，与conv2d方法在官方文档中写的一样，必须保证strides[0]=strides[3]=1，格式为[1, stirde, stride, 1]；
padding依然只有'SAME'和'VALID'；

### Pooling

Pooling层主要的作用是下采样，通过去掉Feature Map中不重要的样本，进一步减少参数数量。Pooling的方法很多，最常用的是Max Pooling。Max Pooling实际上就是在n*n的样本中取最大值，作为采样后的样本值。下图是2*2 max pooling：

![pic3](https://ws4.sinaimg.cn/large/006tKfTcly1g13j25trrdj30hs08b0t9.jpg)

除了Max Pooing之外，常用的还有Mean Pooling——取各样本的平均值。
对于深度为D的Feature Map，各层独立做Pooling，因此Pooling后的深度仍然为D。



## TextCNN

TextCNN 是利用卷积神经网络对文本进行分类的算法，由 Yoon Kim 在 “Convolutional Neural Networks for Sentence Classification” 一文中提出,是2014年的算法.

![pic4](https://ws2.sinaimg.cn/large/006tKfTcly1g13j4hgmhvj31hk0lmdjw.jpg)

![pic5](https://ws2.sinaimg.cn/large/006tKfTcly1g13j4w9a24j30sg0q1q5z.jpg)

Embedding：第一层是图中最左边的7乘5的句子矩阵，每行是词向量，维度=5，这个可以类比为图像中的原始像素点。
Convolution：然后经过 kernel_sizes=(2,3,4) 的一维卷积层，每个kernel_size 有两个输出 channel。
MaxPolling：第三层是一个1-max pooling层，这样不同长度句子经过pooling层之后都能变成定长的表示。
FullConnection and Softmax：最后接一层全连接的softmax 层，输出每个类别的概率。


In [ ]:
# coding: utf-8

import tensorflow as tf


class TCNNConfig(object):
    """CNN配置参数"""

    embedding_dim = 64  # 词向量维度
    seq_length = 600  # 序列长度
    num_classes = 10  # 类别数
    num_filters = 256  # 卷积核数目
    kernel_size = 5  # 卷积核尺寸
    vocab_size = 5000  # 词汇表达小

    hidden_dim = 128  # 全连接层神经元

    dropout_keep_prob = 0.5  # dropout保留比例
    learning_rate = 1e-3  # 学习率

    batch_size = 64  # 每批训练大小
    num_epochs = 10  # 总迭代轮次

    print_per_batch = 100  # 每多少轮输出一次结果
    save_per_batch = 10  # 每多少轮存入tensorboard


class TextCNN(object):
    """文本分类，CNN模型"""

    def __init__(self, config):
        self.config = config

        # 三个待输入的数据
        self.input_x = tf.placeholder(tf.int32, [None, self.config.seq_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, self.config.num_classes], name='input_y')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')

        self.cnn()

    def cnn(self):
        """CNN模型"""
        # 词向量映射
        with tf.device('/cpu:0'):
            embedding = tf.get_variable('embedding', [self.config.vocab_size, self.config.embedding_dim])
            embedding_inputs = tf.nn.embedding_lookup(embedding, self.input_x)

        with tf.name_scope("cnn"):
            # CNN layer
            conv = tf.layers.conv1d(embedding_inputs, self.config.num_filters, self.config.kernel_size, name='conv')
            # global max pooling layer
            gmp = tf.reduce_max(conv, reduction_indices=[1], name='gmp')

        with tf.name_scope("score"):
            # 全连接层，后面接dropout以及relu激活
            fc = tf.layers.dense(gmp, self.config.hidden_dim, name='fc1')
            fc = tf.contrib.layers.dropout(fc, self.keep_prob)
            fc = tf.nn.relu(fc)

            # 分类器
            self.logits = tf.layers.dense(fc, self.config.num_classes, name='fc2')
            self.y_pred_cls = tf.argmax(tf.nn.softmax(self.logits), 1)  # 预测类别

        with tf.name_scope("optimize"):
            # 损失函数，交叉熵
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.input_y)
            self.loss = tf.reduce_mean(cross_entropy)
            # 优化器
            self.optim = tf.train.AdamOptimizer(learning_rate=self.config.learning_rate).minimize(self.loss)

        with tf.name_scope("accuracy"):
            # 准确率
            correct_pred = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)
            self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


train:
![train](https://ws2.sinaimg.cn/large/006tKfTcly1g13k9rs8yqj30ll0hxwi7.jpg)

test:
![test](https://ws2.sinaimg.cn/large/006tKfTcly1g13kap3o5vj30ll0hxq5o.jpg)